In [51]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
from decimal import Decimal, getcontext

getcontext().prec = 80

def win_probability_player_a(elo_a, elo_b):
    elo_a = Decimal(str(elo_a))
    elo_b = Decimal(str(elo_b))
    return Decimal('1') / (Decimal('1') + Decimal('10') ** ((elo_b - elo_a) / Decimal('400')))


def win_probability(monitor_slope, ai_slope, ai_intercept, ai_g_elo, num_steps):
    g_elos = [Decimal(str(g)) for g in np.linspace(0, ai_g_elo, num_steps + 1)]
    monitor_elos = [g * Decimal(str(monitor_slope)) for g in g_elos]
    
    # Convert numpy arrays to lists of Decimals
    g_elos_dec = [Decimal(str(g)) for g in g_elos]
    monitor_elos = [g * Decimal(str(monitor_slope)) for g in g_elos_dec]
    ai_elos = [g * Decimal(str(ai_slope)) + Decimal(str(ai_intercept)) for g in g_elos_dec]
    
    monitor_elos = monitor_elos[:-1]
    ai_elos = ai_elos[1:]
    
    win_probs = [win_probability_player_a(monitor_elos[i], ai_elos[i]) for i in range(len(ai_elos))]


    for p in win_probs:
        assert p < 1

    total_win_prob = Decimal('1')
    for prob in win_probs:
        total_win_prob *= prob
    
    return total_win_prob

monitor_slope = 1
ai_slope = 1
ai_intercept = 10
ai_g_elo = 10
num_steps = 10
print(f"Win probability: {win_probability(monitor_slope, ai_slope, ai_intercept, ai_g_elo, num_steps)}")

Win probability: 0.00070798449137656119684559002644752375114917842954281472854318211229985119355277935


In [52]:
opt_param_dict = {
    'Mafia': {
        'guard': [1.5305382247861692, -622.9317463747565],
        'houdini': [0.8344154600833648, 173.2206142928111],
    },
    'Debate': {
        'guard': [1.0983942546126946, -47.49890598669299],
        'houdini' : [0.04936115584058696, 1080.3983558810423],
    },
    'Backdoor Code': {
        'guard': [0.7192614624976502, 552.0221384375485],
        'houdini': [1.9518243250891987, -1361.61176329289],
    },
    'Wargames': {
        'guard': [1.1666737577004682, -229.07366870912878],
        'houdini': [0.6795738441484533, 422.0053838057133],
    }
}

game_id_list = ['Mafia', 'Debate', 'Backdoor Code', 'Wargames']
game_id_list = ['Mafia', 'Debate', 'Backdoor Code', 'Wargames']
x = np.linspace(0, 2000, 501)
start_elo = 1103

with plt.rc_context({
    'font.family': 'sans-serif',
    'font.size': 5,
    'axes.labelsize': 7.25,
    'axes.titlesize': 7.25,
    'xtick.labelsize': 6,
    'ytick.labelsize': 6,
    'legend.fontsize': 5.5,
}):
    fig, axes = plt.subplots(1, 2, figsize=(5.5, 1.6), gridspec_kw={'wspace': 0.5})

    for game_id in game_id_list:
        guard_slope, guard_int = opt_param_dict[game_id]['guard']
        ai_slope, ai_int = opt_param_dict[game_id]['houdini']
        # precompute intercept shift
        ai_intercept_base = (
            (ai_int - guard_int)
            + start_elo * (ai_slope - guard_slope)
        )

        steps = []
        probs = []
        for ai_g_elo in x:
            best_p = Decimal('0')
            best_n = 1
            for n in range(1, 21):
                p = win_probability(
                    guard_slope,
                    ai_slope,
                    ai_intercept_base,
                    ai_g_elo,
                    n
                )
                if p > best_p:
                    best_p, best_n = p, n

            steps.append(best_n)
            probs.append(best_p)

            # debug at exactly 400
            if ai_g_elo == 400:
                print(
                    f"Game: {game_id}, "
                    f"AI G-Elo: {ai_g_elo}, "
                    f"Optimal Steps: {best_n}, "
                    f"Win Prob: {best_p}"
                )

        axes[0].plot(x, steps, label=game_id)
        axes[1].plot(x, probs, label=game_id)

    # labels & ticks
    axes[0].set_xlabel(r'$\Delta_{\mathrm{general}}$')
    axes[0].set_ylabel('Optimal Steps')
    axes[0].set_yticks(np.arange(0, 15, 2))

    axes[1].set_xlabel(r'$\Delta_{\mathrm{general}}$')
    axes[1].set_ylabel('Win Probability')

    # shared legend above both panels
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(
        handles, labels,
        loc='upper center',
        ncol=len(game_id_list),
        bbox_to_anchor=(0.5, 1.05)
    )
    fig.show()
    
    fig.savefig('./figures/win_prob_real.pdf', bbox_inches='tight')
#    fig.savefig('./figures/win_prob_real.png', bbox_inches='tight')

Game: Mafia, AI G-Elo: 400.0, Optimal Steps: 1, Win Prob: 0.11062359444258422706723677091642544636171686148068322734999450854598927855240270
Game: Debate, AI G-Elo: 400.0, Optimal Steps: 1, Win Prob: 0.51358447510963367244302599275195577983985660970271833404975422849136944649826308
Game: Backdoor Code, AI G-Elo: 400.0, Optimal Steps: 2, Win Prob: 0.27569140767676612969294726409000944977447592600113526805958023052758624981812797
Game: Wargames, AI G-Elo: 400.0, Optimal Steps: 1, Win Prob: 0.097977002063203731666081304638152687439088824498353092261166928338526196691132794
